In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline
pd.set_option('display.max_rows', 10)
# Show the full results
pd.set_option('display.max_column', None)
pd.set_option('display.max_rows', None)

matplotlib.rcParams['figure.figsize'] = (20, 10)

In [3]:
df = pd.read_csv('../Data/Engineered.csv')
df.head()

,location,size,total_sqft,bath,price,price_per_sqft
0,Electronic City Phase II,2,1056.0,2.0,39.07,3699.810606
1,Chikka Tirupathi,4,2600.0,5.0,120.00,4615.384615
2,Uttarahalli,3,1440.0,2.0,62.00,4305.555556
3,Lingadheeranahalli,3,1521.0,3.0,95.00,6245.890861
4,Kothanur,2,1200.0,2.0,51.00,4250.000000
